In [16]:
from dotenv import load_dotenv

load_dotenv()
import os

In [17]:
# wrapper on top of wikipedia API
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [18]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [30]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://python.langchain.com/docs/concepts/agents/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500).split_documents(docs)
embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
# Create a FAISS vector store from the documents
vectordb = FAISS.from_documents(documents, embedding)
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000134F00367D0>, search_kwargs={})

In [20]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever=retriever,name='AgentSearchTool', 
                      description="LangChain Agent Search Tool. Use this tool to search for information related to LangChain agents. ")

In [21]:
retriever_tool.name

'AgentSearchTool'

In [22]:
# Arxiv tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=500)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [23]:
tools = [wiki, retriever_tool, arxiv]

In [24]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\Admin\\Desktop\\LANGCHAIN\\.venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=500)),
 Tool(name='AgentSearchTool', description='LangChain Agent Search Tool. Use this tool to search for information related to LangChain agents. ', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001349293E160>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000013492B81550>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x0000

In [25]:
from langchain_groq import ChatGroq
from langchain.agents import initialize_agent, AgentType

# Step 1: Load Groq LLM
llm = ChatGroq(
    model="llama3-70b-8192",  # or your preferred model
    temperature=0.1,
)



In [27]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor, create_tool_calling_agent
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the following tools to answer the user's questions."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad") # This placeholder is essential for tool calling agents
])

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [31]:
response = agent_executor.invoke({"input": "What are LangChain?"})
print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `AgentSearchTool` with `{'query': 'LangChain'}`


Page Not Found | ü¶úÔ∏èüîó LangChain






Skip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1üí¨SearchPage Not FoundWe could not find what you were looking for.Please contact the owner of the site that linked you to the original URL and let them know their link is broken.CommunityLangChain ForumTwitterGitHubOrganizationPythonJS/TSMoreHomepageBlogYouTubeCopyright ¬© 2025 LangChain, Inc.It seems like the tool call didn't provide a direct answer to your question. LangChain appears to be a platform or organization that provides various tools and resources, including LangGraph, LangChain Hub, LangChain JS/TS, and more. They also have a forum, Twitter, and GitHub presence. However, the specific information abou